In [1]:
import warnings
warnings.simplefilter('ignore')

import gc

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 200)
pd.set_option('float_format', lambda x: '%.6f' % x)

from tqdm import tqdm
tqdm.pandas()

# 合并思路

数据含有三部分，有从属关系，从上至下分为三层：

1. event
2. jet
3. particle

因为同一个 event 的 target 是一样的，因此可以尝试将三层数据合并为一层，用整个 event 来当做输入，进行训练和预测：

1. 将 16 个 particle 合并为一个 jet，合并的顺序为按 abs(particle_px) 从大到小排序，如果超过 16 则截断，小于 16 则补零；
2. 将上一步的 particle data 按 jet_id 与 jet 数据合并；
3. 将 4 个 jet_id 合并为一个 event，合并的顺序为按 abs(jet_px) 从大到小排序，如果超过 4 则截断，小于 4 则补零；
4. 将上一步的 jet data 按 event_id 与 event 数据合并。

上面所述的 4 和 16 均为初步验证设置，后期可以尝试不同的数量。

In [2]:
n_particles = 16
n_jets = 4

In [3]:
# 先读入 particle 数据

df = pd.read_pickle('train_particle_sm.pickle')

In [4]:
# 按 abs(particle_px) 从大到小将每个 jet_id 里面的 particle 数据进行排序

df['particle_px_abs'] = np.abs(df['particle_px'])
df = df.sort_index(by=['jet_id', 'particle_px_abs'], ascending=False).reset_index()

df = df.drop(['particle_px_abs'], axis=1)

In [5]:
df.head()

,index,particle_category,particle_px,particle_py,particle_pz,particle_energy,particle_mass,energy_x,mass_x,mass_yz,particle_speed,jet_id_mass_x_mean,jet_id_mass_x_min,jet_id_mass_x_std,jet_id_mass_yz_mean,jet_id_mass_yz_std,jet_id_energy_x_mean,jet_id_energy_x_min,jet_id_energy_x_std,jet_id_speed_mean,jet_id_speed_min,jet_id_speed_std,jet_id_speed_x_mean,jet_id_speed_x_min,jet_id_speed_x_std,jet_id_px_mean,jet_id_px_min,jet_id_px_std,jet_id_time_x_mean,jet_id_time_x_min,jet_id_time_x_std,jet_id_energy_sum,jet_id_mass_sum,jet_id_speed,jet_id
0,5385914,321,33.437500,-0.466797,0.932129,33.437500,0.493652,33.437500,0.493408,0.015388,11.640625,0.281982,0.000000,0.335693,0.016602,0.024765,10.562500,0.259033,11.078125,5.253906,0.000000,5.187500,5.242188,0.000000,5.191406,10.554688,0.259033,11.078125,1.056641,0.000000,0.926758,338.500000,9.054688,8.648438,fffff8fef65647bf964e2e521f8df138
1,5445737,-211,32.281250,0.308594,0.720703,32.312500,0.139526,32.281250,0.139526,0.003387,21.515625,0.281982,0.000000,0.335693,0.016602,0.024765,10.562500,0.259033,11.078125,5.253906,0.000000,5.187500,5.242188,0.000000,5.191406,10.554688,0.259033,11.078125,1.056641,0.000000,0.926758,338.500000,9.054688,8.648438,fffff8fef65647bf964e2e521f8df138
2,5609036,-321,28.218750,-0.598633,1.284180,28.265625,0.493652,28.234375,0.493164,0.024750,10.703125,0.281982,0.000000,0.335693,0.016602,0.024765,10.562500,0.259033,11.078125,5.253906,0.000000,5.187500,5.242188,0.000000,5.191406,10.554688,0.259033,11.078125,1.056641,0.000000,0.926758,338.500000,9.054688,8.648438,fffff8fef65647bf964e2e521f8df138
3,5431519,22,28.171875,0.832031,1.408203,28.218750,0.000000,28.171875,0.000000,0.000000,0.000000,0.281982,0.000000,0.335693,0.016602,0.024765,10.562500,0.259033,11.078125,5.253906,0.000000,5.187500,5.242188,0.000000,5.191406,10.554688,0.259033,11.078125,1.056641,0.000000,0.926758,338.500000,9.054688,8.648438,fffff8fef65647bf964e2e521f8df138
4,5646787,321,28.078125,-0.428955,1.544922,28.140625,0.493652,28.093750,0.492920,0.028137,10.679688,0.281982,0.000000,0.335693,0.016602,0.024765,10.562500,0.259033,11.078125,5.253906,0.000000,5.187500,5.242188,0.000000,5.191406,10.554688,0.259033,11.078125,1.056641,0.000000,0.926758,338.500000,9.054688,8.648438,fffff8fef65647bf964e2e521f8df138


In [6]:
cols = df.columns.tolist()[1:-1]

In [7]:
# 生成一些辅助变量
# di 记录的是 {jet_id: [当前 jet_id 已加入的 particle 个数, 当前 jet_id 的序号]}

jet = df.jet_id.unique()
num_total_jet = jet.shape[0]

di = dict()
for i, j in tqdm(enumerate(jet.tolist())):
    di[j] = [0, i]

1134555it [00:01, 618522.44it/s]


In [8]:
# 用 np.ndarry 来做处理，使用 dataframe 会相当慢

particle_data = np.zeros(shape=(num_total_jet, n_particles*len(cols)), dtype=np.float16)

for row in tqdm(df.values):
    # 取出该行 particle 的 jet_id
    jet_id = row[-1]
    # n: 当前 jet_id 已加入的 particle 个数
    # i: 当前 jet_id 的序号
    n, i = di[jet_id]
    # 如果当前 jet_id 的 particle 个数少于 n_particles 则加入进来
    if n < n_particles:
        particle_data[i, n*len(cols):(n+1)*len(cols)] = row[1:-1]
        # particle 个数加 1
        di[jet_id][0] += 1

100%|██████████| 24297352/24297352 [01:14<00:00, 328228.35it/s]


In [9]:
del df
gc.collect()

20

In [10]:
# 合并完的 particle 数据

df_particle = pd.DataFrame(particle_data)

In [11]:
df_particle.columns = [f'p{i}_{j}' for i in range(n_particles) for j in cols]
df_particle['jet_id'] = jet

In [12]:
df_particle.head()

,p0_particle_category,p0_particle_px,p0_particle_py,p0_particle_pz,p0_particle_energy,p0_particle_mass,p0_energy_x,p0_mass_x,p0_mass_yz,p0_particle_speed,p0_jet_id_mass_x_mean,p0_jet_id_mass_x_min,p0_jet_id_mass_x_std,p0_jet_id_mass_yz_mean,p0_jet_id_mass_yz_std,p0_jet_id_energy_x_mean,p0_jet_id_energy_x_min,p0_jet_id_energy_x_std,p0_jet_id_speed_mean,p0_jet_id_speed_min,p0_jet_id_speed_std,p0_jet_id_speed_x_mean,p0_jet_id_speed_x_min,p0_jet_id_speed_x_std,p0_jet_id_px_mean,p0_jet_id_px_min,p0_jet_id_px_std,p0_jet_id_time_x_mean,p0_jet_id_time_x_min,p0_jet_id_time_x_std,p0_jet_id_energy_sum,p0_jet_id_mass_sum,p0_jet_id_speed,p1_particle_category,p1_particle_px,p1_particle_py,p1_particle_pz,p1_particle_energy,p1_particle_mass,p1_energy_x,p1_mass_x,p1_mass_yz,p1_particle_speed,p1_jet_id_mass_x_mean,p1_jet_id_mass_x_min,p1_jet_id_mass_x_std,p1_jet_id_mass_yz_mean,p1_jet_id_mass_yz_std,p1_jet_id_energy_x_mean,p1_jet_id_energy_x_min,p1_jet_id_energy_x_std,p1_jet_id_speed_mean,p1_jet_id_speed_min,p1_jet_id_speed_std,p1_jet_id_speed_x_mean,p1_jet_id_speed_x_min,p1_jet_id_speed_x_std,p1_jet_id_px_mean,p1_jet_id_px_min,p1_jet_id_px_std,p1_jet_id_time_x_mean,p1_jet_id_time_x_min,p1_jet_id_time_x_std,p1_jet_id_energy_sum,p1_jet_id_mass_sum,p1_jet_id_speed,p2_particle_category,p2_particle_px,p2_particle_py,p2_particle_pz,p2_particle_energy,p2_particle_mass,p2_energy_x,p2_mass_x,p2_mass_yz,p2_particle_speed,p2_jet_id_mass_x_mean,p2_jet_id_mass_x_min,p2_jet_id_mass_x_std,p2_jet_id_mass_yz_mean,p2_jet_id_mass_yz_std,p2_jet_id_energy_x_mean,p2_jet_id_energy_x_min,p2_jet_id_energy_x_std,p2_jet_id_speed_mean,p2_jet_id_speed_min,p2_jet_id_speed_std,p2_jet_id_speed_x_mean,p2_jet_id_speed_x_min,p2_jet_id_speed_x_std,p2_jet_id_px_mean,p2_jet_id_px_min,p2_jet_id_px_std,p2_jet_id_time_x_mean,p2_jet_id_time_x_min,p2_jet_id_time_x_std,p2_jet_id_energy_sum,p2_jet_id_mass_sum,p2_jet_id_speed,p3_particle_category,p3_particle_px,p3_particle_py,p3_particle_pz,p3_particle_energy,p3_particle_mass,p3_energy_x,p3_mass_x,p3_mass_yz,p3_particle_speed,p3_jet_id_mass_x_mean,p3_jet_id_mass_x_min,p3_jet_id_mass_x_std,p3_jet_id_mass_yz_mean,p3_jet_id_mass_yz_std,p3_jet_id_energy_x_mean,p3_jet_id_energy_x_min,p3_jet_id_energy_x_std,p3_jet_id_speed_mean,p3_jet_id_speed_min,p3_jet_id_speed_std,p3_jet_id_speed_x_mean,p3_jet_id_speed_x_min,p3_jet_id_speed_x_std,p3_jet_id_px_mean,p3_jet_id_px_min,p3_jet_id_px_std,p3_jet_id_time_x_mean,p3_jet_id_time_x_min,p3_jet_id_time_x_std,p3_jet_id_energy_sum,p3_jet_id_mass_sum,p3_jet_id_speed,p4_particle_category,p4_particle_px,p4_particle_py,p4_particle_pz,p4_particle_energy,p4_particle_mass,p4_energy_x,p4_mass_x,p4_mass_yz,p4_particle_speed,p4_jet_id_mass_x_mean,p4_jet_id_mass_x_min,p4_jet_id_mass_x_std,p4_jet_id_mass_yz_mean,p4_jet_id_mass_yz_std,p4_jet_id_energy_x_mean,p4_jet_id_energy_x_min,p4_jet_id_energy_x_std,p4_jet_id_speed_mean,p4_jet_id_speed_min,p4_jet_id_speed_std,p4_jet_id_speed_x_mean,p4_jet_id_speed_x_min,p4_jet_id_speed_x_std,p4_jet_id_px_mean,p4_jet_id_px_min,p4_jet_id_px_std,p4_jet_id_time_x_mean,p4_jet_id_time_x_min,p4_jet_id_time_x_std,p4_jet_id_energy_sum,p4_jet_id_mass_sum,p4_jet_id_speed,p5_particle_category,p5_particle_px,p5_particle_py,p5_particle_pz,p5_particle_energy,p5_particle_mass,p5_energy_x,p5_mass_x,p5_mass_yz,p5_particle_speed,p5_jet_id_mass_x_mean,p5_jet_id_mass_x_min,p5_jet_id_mass_x_std,p5_jet_id_mass_yz_mean,p5_jet_id_mass_yz_std,p5_jet_id_energy_x_mean,p5_jet_id_energy_x_min,p5_jet_id_energy_x_std,p5_jet_id_speed_mean,p5_jet_id_speed_min,p5_jet_id_speed_std,p5_jet_id_speed_x_mean,p5_jet_id_speed_x_min,p5_jet_id_speed_x_std,p5_jet_id_px_mean,p5_jet_id_px_min,p5_jet_id_px_std,p5_jet_id_time_x_mean,p5_jet_id_time_x_min,p5_jet_id_time_x_std,p5_jet_id_energy_sum,p5_jet_id_mass_sum,p5_jet_id_speed,p6_particle_category,p6_particle_px,p6_particle_py,p6_particle_pz,p6_particle_energy,p6_particle_mass,p6_energy_x,p6_mass_x,p6_mass_yz,p6_particle_speed,p6_jet_id_mass_x_mean,p6_jet_id_mass_x_min,p6_jet_id_m

In [13]:
for i in range(n_particles):
    df_particle[f'p{i}_particle_category'] = df_particle[f'p{i}_particle_category'].astype('int')

In [14]:
# 读入 jet 数据

# train_jet = pd.read_csv('jet_complex_data/complex_train_R04_jet.csv')
train_jet = pd.read_pickle('train_jet.float16.pickle')

In [15]:
# particle 数据与 jet 数据合并

df = pd.merge(train_jet, df_particle, on=['jet_id'])

In [16]:
del df_particle, particle_data

gc.collect()

20

In [17]:
for col in tqdm(df.columns.tolist()):
    if df[col].dtypes == 'float64':
        df[col] = df[col].astype('float16')

100%|██████████| 855/855 [00:00<00:00, 24414.71it/s]


In [18]:
gc.collect()

20

In [19]:
# 按 abs(jet_px) 从大到小将每个 event_id 里面的 jet 数据进行排序

df['jet_px_abs'] = np.abs(df['jet_px'])
df = df.sort_index(by=['event_id', 'jet_px_abs'], ascending=False).reset_index()

df = df.drop(['jet_px_abs'], axis=1)

In [20]:
# 需要合并的特征

cols = [i for i in df.columns if i not in ['index', 'jet_id', 'event_id']]

In [21]:
df = df.reindex(columns=cols+['index', 'jet_id', 'event_id'])

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1134555 entries, 0 to 1134554
Columns: 856 entries, jet_energy to event_id
dtypes: float16(833), int16(4), int64(17), object(2)
memory usage: 1.9+ GB


In [23]:
df.to_pickle('train_jet_temp.pickle')

# 在这里重启并往下执行

In [1]:
import warnings
warnings.simplefilter('ignore')

import gc

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 200)
pd.set_option('float_format', lambda x: '%.6f' % x)

from tqdm import tqdm
tqdm.pandas()

In [2]:
n_particles = 16
n_jets = 4

In [3]:
df = pd.read_pickle('train_jet_temp.pickle')

In [4]:
# 需要合并的特征

cols = [i for i in df.columns if i not in ['index', 'jet_id', 'event_id']]

In [5]:
# 生成一些辅助变量
# di 记录的是 {event_id: [当前 event_id 已加入的 jet 个数, 当前 event_id 的序号]}

event = df.event_id.unique()
num_total_event = event.shape[0]

di = dict()
for i, j in tqdm(enumerate(event.tolist())):
    di[j] = [0, i]

371377it [00:00, 747699.08it/s]


In [6]:
df.head()

,jet_energy,jet_mass,jet_px,jet_py,jet_pz,label,number_of_particles_in_this_jet,jet_distance,xy_dis,yz_dis,zx_dis,x_div_dist,y_div_dist,z_div_dist,xy_div_dist,yz_div_dist,zx_div_dist,energy_x,energy_y,energy_z,energy_xy,energy_yz,energy_zx,mass_x,mass_y,mass_z,mass_xy,mass_yz,mass_zx,angle_xy,angle_yx,angle_yz,angle_zy,angle_zx,angle_xz,mean_particles_mass,mean_particles_energy,jet_speed,speed_x,speed_y,speed_z,speed_xy,speed_yz,speed_zx,time_dis,time_x,time_y,time_z,time_xy,time_yz,time_zx,jet_mv,mv_x,mv_y,mv_z,mv_xy,mv_yz,mv_zx,particle_mv,particle_mv_x,particle_mv_y,particle_mv_z,particle_mv_xy,particle_mv_yz,particle_mv_zx,event_id_count,event_id_number_particles_max,event_id_number_particles_mean,event_id_number_particles_min,event_id_number_particles_std,event_id_mass_max,event_id_mass_mean,event_id_mass_min,event_id_mass_std,event_id_energy_max,event_id_energy_mean,event_id_energy_min,event_id_energy_std,event_id_mass_x_max,event_id_mass_x_mean,event_id_mass_x_min,event_id_mass_x_std,event_id_mass_y_max,event_id_mass_y_mean,event_id_mass_y_min,event_id_mass_y_std,event_id_mass_z_max,event_id_mass_z_mean,event_id_mass_z_min,event_id_mass_z_std,event_id_mass_xy_max,event_id_mass_xy_mean,event_id_mass_xy_min,event_id_mass_xy_std,event_id_mass_yz_max,event_id_mass_yz_mean,event_id_mass_yz_min,event_id_mass_yz_std,event_id_mass_zx_max,event_id_mass_zx_mean,event_id_mass_zx_min,event_id_mass_zx_std,event_id_energy_x_max,event_id_energy_x_mean,event_id_energy_x_min,event_id_energy_x_std,event_id_energy_y_max,event_id_energy_y_mean,event_id_energy_y_min,event_id_energy_y_std,event_id_energy_z_max,event_id_energy_z_mean,event_id_energy_z_min,event_id_energy_z_std,event_id_energy_xy_max,event_id_energy_xy_mean,event_id_energy_xy_min,event_id_energy_xy_std,event_id_energy_yz_max,event_id_energy_yz_mean,event_id_energy_yz_min,event_id_energy_yz_std,event_id_energy_zx_max,event_id_energy_zx_mean,event_id_energy_zx_min,event_id_energy_zx_std,event_id_particles_mass_max,event_id_particles_mass_mean,event_id_particles_mass_min,event_id_particles_mass_std,event_id_particles_energy_max,event_id_particles_energy_mean,event_id_particles_energy_min,event_id_particles_energy_std,event_id_distance_max,event_id_distance_mean,event_id_distance_min,event_id_distance_std,event_id_xy_dis_max,event_id_xy_dis_mean,event_id_xy_dis_min,event_id_xy_dis_std,event_id_yz_dis_max,event_id_yz_dis_mean,event_id_yz_dis_min,event_id_yz_dis_std,event_id_zx_dis_max,event_id_zx_dis_mean,event_id_zx_dis_min,event_id_zx_dis_std,event_id_x_div_dist_max,event_id_x_div_dist_mean,event_id_x_div_dist_min,event_id_x_div_dist_std,event_id_y_div_dist_max,event_id_y_div_dist_mean,event_id_y_div_dist_min,event_id_y_div_dist_std,event_id_z_div_dist_max,event_id_z_div_dist_mean,event_id_z_div_dist_min,event_id_z_div_dist_std,event_id_xy_div_dist_max,event_id_xy_div_dist_mean,event_id_xy_div_dist_min,event_id_xy_div_dist_std,event_id_yz_div_dist_max,event_id_yz_div_dist_mean,event_id_yz_div_dist_min,event_id_yz_div_dist_std,event_id_zx_div_dist_max,event_id_zx_div_dist_mean,event_id_zx_div_dist_min,event_id_zx_div_dist_std,event_id_speed_max,event_id_speed_mean,event_id_speed_min,event_id_speed_std,event_id_speed_x_max,event_id_speed_x_mean,event_id_speed_x_min,event_id_speed_x_std,event_id_speed_y_max,event_id_speed_y_mean,event_id_speed_y_min,event_id_speed_y_std,event_id_speed_z_max,event_id_speed_z_mean,event_id_speed_z_min,event_id_speed_z_std,event_id_speed_xy_max,event_id_speed_xy_mean,event_id_speed_xy_min,event_id_speed_xy_std,event_id_speed_yz_max,event_id_speed_yz_mean,event_id_speed_yz_min,event_id_speed_yz_std,event_id_speed_zx_max,event_id_speed_zx_mean,event_id_speed_zx_min,event_id_speed_zx_std,event_id_px_max,event_id_px_mean,event_id_px_min,event_id_px_std,event_id_py_max,event_id_py_mean,event_id_py_min,event_id_py_std,event_id_pz_max,event_id_pz_mean,event_id_pz_min,event_id_pz_std,event_id_angle_xy_max,event_id_angle_xy_mean,event_id_angle_xy_min,event_id_a

In [7]:
gc.collect()

20

In [8]:
# 用 np.ndarry 来做处理，使用 dataframe 会相当慢

jet_data = np.zeros(shape=(num_total_event, n_jets*len(cols)), dtype=np.float16)

for row in tqdm(df.values):
    # 取出该行 jet 的 event_id
    event_id = row[-1]
    # n: 当前 event_id 已加入的 jet 个数
    # i: 当前 event_id 的序号
    n, i = di[event_id]
    # 如果当前 event_id 的 jet 个数少于 4 则加入进来
    if n < n_jets:
        jet_data[i, n*len(cols):(n+1)*len(cols)] = row[:-3]
        # particle 个数加 1
        di[event_id][0] += 1

100%|██████████| 1134555/1134555 [04:32<00:00, 4164.56it/s]


In [9]:
del df
gc.collect()

40

In [10]:
# 合并完的 jet 数据

df_jet = pd.DataFrame(jet_data)

In [11]:
df_jet.columns = np.array([[f'j{j}_{i}' for i in cols if i not in ['index', 'jet_id', 'event_id']] for j in range(4)]).flatten().tolist()
df_jet['event_id'] = event

In [12]:
df_jet.head()

,j0_jet_energy,j0_jet_mass,j0_jet_px,j0_jet_py,j0_jet_pz,j0_label,j0_number_of_particles_in_this_jet,j0_jet_distance,j0_xy_dis,j0_yz_dis,j0_zx_dis,j0_x_div_dist,j0_y_div_dist,j0_z_div_dist,j0_xy_div_dist,j0_yz_div_dist,j0_zx_div_dist,j0_energy_x,j0_energy_y,j0_energy_z,j0_energy_xy,j0_energy_yz,j0_energy_zx,j0_mass_x,j0_mass_y,j0_mass_z,j0_mass_xy,j0_mass_yz,j0_mass_zx,j0_angle_xy,j0_angle_yx,j0_angle_yz,j0_angle_zy,j0_angle_zx,j0_angle_xz,j0_mean_particles_mass,j0_mean_particles_energy,j0_jet_speed,j0_speed_x,j0_speed_y,j0_speed_z,j0_speed_xy,j0_speed_yz,j0_speed_zx,j0_time_dis,j0_time_x,j0_time_y,j0_time_z,j0_time_xy,j0_time_yz,j0_time_zx,j0_jet_mv,j0_mv_x,j0_mv_y,j0_mv_z,j0_mv_xy,j0_mv_yz,j0_mv_zx,j0_particle_mv,j0_particle_mv_x,j0_particle_mv_y,j0_particle_mv_z,j0_particle_mv_xy,j0_particle_mv_yz,j0_particle_mv_zx,j0_event_id_count,j0_event_id_number_particles_max,j0_event_id_number_particles_mean,j0_event_id_number_particles_min,j0_event_id_number_particles_std,j0_event_id_mass_max,j0_event_id_mass_mean,j0_event_id_mass_min,j0_event_id_mass_std,j0_event_id_energy_max,j0_event_id_energy_mean,j0_event_id_energy_min,j0_event_id_energy_std,j0_event_id_mass_x_max,j0_event_id_mass_x_mean,j0_event_id_mass_x_min,j0_event_id_mass_x_std,j0_event_id_mass_y_max,j0_event_id_mass_y_mean,j0_event_id_mass_y_min,j0_event_id_mass_y_std,j0_event_id_mass_z_max,j0_event_id_mass_z_mean,j0_event_id_mass_z_min,j0_event_id_mass_z_std,j0_event_id_mass_xy_max,j0_event_id_mass_xy_mean,j0_event_id_mass_xy_min,j0_event_id_mass_xy_std,j0_event_id_mass_yz_max,j0_event_id_mass_yz_mean,j0_event_id_mass_yz_min,j0_event_id_mass_yz_std,j0_event_id_mass_zx_max,j0_event_id_mass_zx_mean,j0_event_id_mass_zx_min,j0_event_id_mass_zx_std,j0_event_id_energy_x_max,j0_event_id_energy_x_mean,j0_event_id_energy_x_min,j0_event_id_energy_x_std,j0_event_id_energy_y_max,j0_event_id_energy_y_mean,j0_event_id_energy_y_min,j0_event_id_energy_y_std,j0_event_id_energy_z_max,j0_event_id_energy_z_mean,j0_event_id_energy_z_min,j0_event_id_energy_z_std,j0_event_id_energy_xy_max,j0_event_id_energy_xy_mean,j0_event_id_energy_xy_min,j0_event_id_energy_xy_std,j0_event_id_energy_yz_max,j0_event_id_energy_yz_mean,j0_event_id_energy_yz_min,j0_event_id_energy_yz_std,j0_event_id_energy_zx_max,j0_event_id_energy_zx_mean,j0_event_id_energy_zx_min,j0_event_id_energy_zx_std,j0_event_id_particles_mass_max,j0_event_id_particles_mass_mean,j0_event_id_particles_mass_min,j0_event_id_particles_mass_std,j0_event_id_particles_energy_max,j0_event_id_particles_energy_mean,j0_event_id_particles_energy_min,j0_event_id_particles_energy_std,j0_event_id_distance_max,j0_event_id_distance_mean,j0_event_id_distance_min,j0_event_id_distance_std,j0_event_id_xy_dis_max,j0_event_id_xy_dis_mean,j0_event_id_xy_dis_min,j0_event_id_xy_dis_std,j0_event_id_yz_dis_max,j0_event_id_yz_dis_mean,j0_event_id_yz_dis_min,j0_event_id_yz_dis_std,j0_event_id_zx_dis_max,j0_event_id_zx_dis_mean,j0_event_id_zx_dis_min,j0_event_id_zx_dis_std,j0_event_id_x_div_dist_max,j0_event_id_x_div_dist_mean,j0_event_id_x_div_dist_min,j0_event_id_x_div_dist_std,j0_event_id_y_div_dist_max,j0_event_id_y_div_dist_mean,j0_event_id_y_div_dist_min,j0_event_id_y_div_dist_std,j0_event_id_z_div_dist_max,j0_event_id_z_div_dist_mean,j0_event_id_z_div_dist_min,j0_event_id_z_div_dist_std,j0_event_id_xy_div_dist_max,j0_event_id_xy_div_dist_mean,j0_event_id_xy_div_dist_min,j0_event_id_xy_div_dist_std,j0_event_id_yz_div_dist_max,j0_event_id_yz_div_dist_mean,j0_event_id_yz_div_dist_min,j0_event_id_yz_div_dist_std,j0_event_id_zx_div_dist_max,j0_event_id_zx_div_dist_mean,j0_event_id_zx_div_dist_min,j0_event_id_zx_div_dist_std,j0_event_id_speed_max,j0_event_id_speed_mean,j0_event_id_speed_min,j0_event_id_speed_std,j0_event_id_speed_x_max,j0_event_id_speed_x_mean,j0_event_id_speed_x_min,j0_event_id_speed_x_std,j0_event_id_speed_y_max,j0_event_id_speed_y_mean,j0_event_id_speed_y_min,j0_event_id_speed_y_std,j0_event_id_speed_z_max,j0_event_id_speed_z_mean,j0_event_id_speed

In [13]:
del jet_data

gc.collect()

20

In [14]:
# 读入 event 数据

train_event = pd.read_csv('jet_complex_data/complex_train_R04_event.csv')

In [15]:
train_event.shape, df_jet.shape

((371377, 2), (371377, 3413))

In [16]:
df = pd.merge(df_jet, train_event, on=['event_id'])

In [17]:
gc.collect()

94

In [18]:
df['label'] = df['j0_label']
df = df.drop([f'j{i}_label' for i in range(4)], axis=1)
df['label'] = df['label'].astype('int')

In [19]:
df.shape

(371377, 3411)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 371377 entries, 0 to 371376
Columns: 3411 entries, j0_jet_energy to label
dtypes: float16(3408), int64(2), object(1)
memory usage: 2.4+ GB


In [21]:
df.head()

,j0_jet_energy,j0_jet_mass,j0_jet_px,j0_jet_py,j0_jet_pz,j0_number_of_particles_in_this_jet,j0_jet_distance,j0_xy_dis,j0_yz_dis,j0_zx_dis,j0_x_div_dist,j0_y_div_dist,j0_z_div_dist,j0_xy_div_dist,j0_yz_div_dist,j0_zx_div_dist,j0_energy_x,j0_energy_y,j0_energy_z,j0_energy_xy,j0_energy_yz,j0_energy_zx,j0_mass_x,j0_mass_y,j0_mass_z,j0_mass_xy,j0_mass_yz,j0_mass_zx,j0_angle_xy,j0_angle_yx,j0_angle_yz,j0_angle_zy,j0_angle_zx,j0_angle_xz,j0_mean_particles_mass,j0_mean_particles_energy,j0_jet_speed,j0_speed_x,j0_speed_y,j0_speed_z,j0_speed_xy,j0_speed_yz,j0_speed_zx,j0_time_dis,j0_time_x,j0_time_y,j0_time_z,j0_time_xy,j0_time_yz,j0_time_zx,j0_jet_mv,j0_mv_x,j0_mv_y,j0_mv_z,j0_mv_xy,j0_mv_yz,j0_mv_zx,j0_particle_mv,j0_particle_mv_x,j0_particle_mv_y,j0_particle_mv_z,j0_particle_mv_xy,j0_particle_mv_yz,j0_particle_mv_zx,j0_event_id_count,j0_event_id_number_particles_max,j0_event_id_number_particles_mean,j0_event_id_number_particles_min,j0_event_id_number_particles_std,j0_event_id_mass_max,j0_event_id_mass_mean,j0_event_id_mass_min,j0_event_id_mass_std,j0_event_id_energy_max,j0_event_id_energy_mean,j0_event_id_energy_min,j0_event_id_energy_std,j0_event_id_mass_x_max,j0_event_id_mass_x_mean,j0_event_id_mass_x_min,j0_event_id_mass_x_std,j0_event_id_mass_y_max,j0_event_id_mass_y_mean,j0_event_id_mass_y_min,j0_event_id_mass_y_std,j0_event_id_mass_z_max,j0_event_id_mass_z_mean,j0_event_id_mass_z_min,j0_event_id_mass_z_std,j0_event_id_mass_xy_max,j0_event_id_mass_xy_mean,j0_event_id_mass_xy_min,j0_event_id_mass_xy_std,j0_event_id_mass_yz_max,j0_event_id_mass_yz_mean,j0_event_id_mass_yz_min,j0_event_id_mass_yz_std,j0_event_id_mass_zx_max,j0_event_id_mass_zx_mean,j0_event_id_mass_zx_min,j0_event_id_mass_zx_std,j0_event_id_energy_x_max,j0_event_id_energy_x_mean,j0_event_id_energy_x_min,j0_event_id_energy_x_std,j0_event_id_energy_y_max,j0_event_id_energy_y_mean,j0_event_id_energy_y_min,j0_event_id_energy_y_std,j0_event_id_energy_z_max,j0_event_id_energy_z_mean,j0_event_id_energy_z_min,j0_event_id_energy_z_std,j0_event_id_energy_xy_max,j0_event_id_energy_xy_mean,j0_event_id_energy_xy_min,j0_event_id_energy_xy_std,j0_event_id_energy_yz_max,j0_event_id_energy_yz_mean,j0_event_id_energy_yz_min,j0_event_id_energy_yz_std,j0_event_id_energy_zx_max,j0_event_id_energy_zx_mean,j0_event_id_energy_zx_min,j0_event_id_energy_zx_std,j0_event_id_particles_mass_max,j0_event_id_particles_mass_mean,j0_event_id_particles_mass_min,j0_event_id_particles_mass_std,j0_event_id_particles_energy_max,j0_event_id_particles_energy_mean,j0_event_id_particles_energy_min,j0_event_id_particles_energy_std,j0_event_id_distance_max,j0_event_id_distance_mean,j0_event_id_distance_min,j0_event_id_distance_std,j0_event_id_xy_dis_max,j0_event_id_xy_dis_mean,j0_event_id_xy_dis_min,j0_event_id_xy_dis_std,j0_event_id_yz_dis_max,j0_event_id_yz_dis_mean,j0_event_id_yz_dis_min,j0_event_id_yz_dis_std,j0_event_id_zx_dis_max,j0_event_id_zx_dis_mean,j0_event_id_zx_dis_min,j0_event_id_zx_dis_std,j0_event_id_x_div_dist_max,j0_event_id_x_div_dist_mean,j0_event_id_x_div_dist_min,j0_event_id_x_div_dist_std,j0_event_id_y_div_dist_max,j0_event_id_y_div_dist_mean,j0_event_id_y_div_dist_min,j0_event_id_y_div_dist_std,j0_event_id_z_div_dist_max,j0_event_id_z_div_dist_mean,j0_event_id_z_div_dist_min,j0_event_id_z_div_dist_std,j0_event_id_xy_div_dist_max,j0_event_id_xy_div_dist_mean,j0_event_id_xy_div_dist_min,j0_event_id_xy_div_dist_std,j0_event_id_yz_div_dist_max,j0_event_id_yz_div_dist_mean,j0_event_id_yz_div_dist_min,j0_event_id_yz_div_dist_std,j0_event_id_zx_div_dist_max,j0_event_id_zx_div_dist_mean,j0_event_id_zx_div_dist_min,j0_event_id_zx_div_dist_std,j0_event_id_speed_max,j0_event_id_speed_mean,j0_event_id_speed_min,j0_event_id_speed_std,j0_event_id_speed_x_max,j0_event_id_speed_x_mean,j0_event_id_speed_x_min,j0_event_id_speed_x_std,j0_event_id_speed_y_max,j0_event_id_speed_y_mean,j0_event_id_speed_y_min,j0_event_id_speed_y_std,j0_event_id_speed_z_max,j0_event_id_speed_z_mean,j0_event_id_speed_z_min,j0

In [22]:
del df_jet, train_event

gc.collect()

41

In [23]:
df.to_csv('train_04j16p_fe_v1.csv', index=False)

In [24]:
for i, col in enumerate(df.columns.tolist()):
    print(i, col)

0 j0_jet_energy
1 j0_jet_mass
2 j0_jet_px
3 j0_jet_py
4 j0_jet_pz
5 j0_number_of_particles_in_this_jet
6 j0_jet_distance
7 j0_xy_dis
8 j0_yz_dis
9 j0_zx_dis
10 j0_x_div_dist
11 j0_y_div_dist
12 j0_z_div_dist
13 j0_xy_div_dist
14 j0_yz_div_dist
15 j0_zx_div_dist
16 j0_energy_x
17 j0_energy_y
18 j0_energy_z
19 j0_energy_xy
20 j0_energy_yz
21 j0_energy_zx
22 j0_mass_x
23 j0_mass_y
24 j0_mass_z
25 j0_mass_xy
26 j0_mass_yz
27 j0_mass_zx
28 j0_angle_xy
29 j0_angle_yx
30 j0_angle_yz
31 j0_angle_zy
32 j0_angle_zx
33 j0_angle_xz
34 j0_mean_particles_mass
35 j0_mean_particles_energy
36 j0_jet_speed
37 j0_speed_x
38 j0_speed_y
39 j0_speed_z
40 j0_speed_xy
41 j0_speed_yz
42 j0_speed_zx
43 j0_time_dis
44 j0_time_x
45 j0_time_y
46 j0_time_z
47 j0_time_xy
48 j0_time_yz
49 j0_time_zx
50 j0_jet_mv
51 j0_mv_x
52 j0_mv_y
53 j0_mv_z
54 j0_mv_xy
55 j0_mv_yz
56 j0_mv_zx
57 j0_particle_mv
58 j0_particle_mv_x
59 j0_particle_mv_y
60 j0_particle_mv_z
61 j0_particle_mv_xy
62 j0_particle_mv_yz
63 j0_particle_mv_z

1485 j1_p9_jet_id_mass_x_std
1486 j1_p9_jet_id_mass_yz_mean
1487 j1_p9_jet_id_mass_yz_std
1488 j1_p9_jet_id_energy_x_mean
1489 j1_p9_jet_id_energy_x_min
1490 j1_p9_jet_id_energy_x_std
1491 j1_p9_jet_id_speed_mean
1492 j1_p9_jet_id_speed_min
1493 j1_p9_jet_id_speed_std
1494 j1_p9_jet_id_speed_x_mean
1495 j1_p9_jet_id_speed_x_min
1496 j1_p9_jet_id_speed_x_std
1497 j1_p9_jet_id_px_mean
1498 j1_p9_jet_id_px_min
1499 j1_p9_jet_id_px_std
1500 j1_p9_jet_id_time_x_mean
1501 j1_p9_jet_id_time_x_min
1502 j1_p9_jet_id_time_x_std
1503 j1_p9_jet_id_energy_sum
1504 j1_p9_jet_id_mass_sum
1505 j1_p9_jet_id_speed
1506 j1_p10_particle_category
1507 j1_p10_particle_px
1508 j1_p10_particle_py
1509 j1_p10_particle_pz
1510 j1_p10_particle_energy
1511 j1_p10_particle_mass
1512 j1_p10_energy_x
1513 j1_p10_mass_x
1514 j1_p10_mass_yz
1515 j1_p10_particle_speed
1516 j1_p10_jet_id_mass_x_mean
1517 j1_p10_jet_id_mass_x_min
1518 j1_p10_jet_id_mass_x_std
1519 j1_p10_jet_id_mass_yz_mean
1520 j1_p10_jet_id_mass_yz_std

3209 j3_p9_jet_id_speed
3210 j3_p10_particle_category
3211 j3_p10_particle_px
3212 j3_p10_particle_py
3213 j3_p10_particle_pz
3214 j3_p10_particle_energy
3215 j3_p10_particle_mass
3216 j3_p10_energy_x
3217 j3_p10_mass_x
3218 j3_p10_mass_yz
3219 j3_p10_particle_speed
3220 j3_p10_jet_id_mass_x_mean
3221 j3_p10_jet_id_mass_x_min
3222 j3_p10_jet_id_mass_x_std
3223 j3_p10_jet_id_mass_yz_mean
3224 j3_p10_jet_id_mass_yz_std
3225 j3_p10_jet_id_energy_x_mean
3226 j3_p10_jet_id_energy_x_min
3227 j3_p10_jet_id_energy_x_std
3228 j3_p10_jet_id_speed_mean
3229 j3_p10_jet_id_speed_min
3230 j3_p10_jet_id_speed_std
3231 j3_p10_jet_id_speed_x_mean
3232 j3_p10_jet_id_speed_x_min
3233 j3_p10_jet_id_speed_x_std
3234 j3_p10_jet_id_px_mean
3235 j3_p10_jet_id_px_min
3236 j3_p10_jet_id_px_std
3237 j3_p10_jet_id_time_x_mean
3238 j3_p10_jet_id_time_x_min
3239 j3_p10_jet_id_time_x_std
3240 j3_p10_jet_id_energy_sum
3241 j3_p10_jet_id_mass_sum
3242 j3_p10_jet_id_speed
3243 j3_p11_particle_category
3244 j3_p11_parti